# Labeling ESA tables with VEC labels

In [3]:
import pandas as pd
import numpy as np
import os
from snorkel.labeling import labeling_function
from snorkel.labeling import LabelingFunction
from snorkel.labeling.lf.nlp import nlp_labeling_function
from snorkel.preprocess import preprocessor
from snorkel.labeling import PandasLFApplier
from snorkel.labeling.apply.dask import DaskLFApplier
from sklearn.model_selection import train_test_split
import spacy
from spacy.matcher import Matcher
import numpy as np

from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from transformers import BertTokenizerFast, BertModel, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from transformers import AutoModelForSequenceClassification

import pytorch_lightning as pl
from pytorch_lightning.metrics.functional import accuracy, f1, auroc
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, BackboneFinetuning, ModelPruning
RANDOM_SEED = 42
pl.seed_everything(RANDOM_SEED)

# weights and biases
import wandb

# lightning plus wandb
from pytorch_lightning.loggers import WandbLogger

Global seed set to 42


In [4]:
!nvidia-smi

Mon Jun  7 20:10:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.89       Driver Version: 465.89       CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Quadro R... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   60C    P8     9W /  N/A |   1223MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
esa_df = pd.read_csv('./data/esa_index_with_table_text_no_labels.csv')

In [22]:
esa_df.drop(columns=['Unnamed: 0', 'Index'], inplace=True)

In [23]:
esa_df.head()

,Title,Content Type,Application Name,Application Short Name,Application Filing Date,Company Name,Commodity,File Name,ESA Folder URL,Document Number,...,PDF Page Number,PDF Page Count,PDF Size,PDF Outline,Download folder name,Zipped Project Link,Missing CSV,CSV Filename,csvFileName,csvText
0,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,14,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-3-summary-of-aquatics-field-wor...,1059614_14_lattice-v_1.csv,Survey Date Aboriginal Communities De...
1,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,17,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_17_lattice-v_1.csv,Field Site No Name Legal Location ...
2,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,18,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_18_lattice-v_1.csv,Field Site No Name Legal Location ...
3,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,19,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_19_lattice-v_1.csv,Field Site No Name Legal Location ...
4,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Table,Application for North Montney Project,North Montney,11/8/2013,NOVA Gas Transmission Ltd.,Gas,B2-16 ESA_Appendix_G_Part1of4 (A3Q6H2),https://apps.cer-rec.gc.ca/REGDOCS/Item/LoadRe...,A3Q6H2,...,20,48.0,5.87,No,nrthmntn,http://www.cer-rec.gc.ca/esa-ees/nrthmntn.zip,False,nrthmntn_table-4-summary-of-watercourse-crossi...,1059614_20_lattice-v_1.csv,Field Site No Name Legal Location ...


In [24]:
esa_df['csvText'] = esa_df['csvText'].replace(' +', ' ', regex=True)

In [25]:
esa_df['csvText'].head()

0     Survey Date Aboriginal Communities Detail Jul...
1     Field Site No Name Legal Location UTM Coordin...
2     Field Site No Name Legal Location UTM Coordin...
3     Field Site No Name Legal Location UTM Coordin...
4     Field Site No Name Legal Location UTM Coordin...
Name: csvText, dtype: object

In [29]:
snorkel_df = esa_df[['Title', 'csvText']]
snorkel_df.rename(columns={"csvText": "text"}, inplace=True)
snorkel_df['label'] = -1

C:\Users\ThibJacq\Anaconda3\lib\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-29-b648c9c9aca6>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snorkel_df['label'] = -1


In [30]:
snorkel_df

,Title,text,label
0,TABLE 3 SUMMARY OF AQUATICS FIELD WORK AND ABO...,Survey Date Aboriginal Communities Detail Jul...,-1
1,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Field Site No Name Legal Location UTM Coordin...,-1
2,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Field Site No Name Legal Location UTM Coordin...,-1
3,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Field Site No Name Legal Location UTM Coordin...,-1
4,TABLE 4 SUMMARY OF WATERCOURSE CROSSINGS ALONG...,Field Site No Name Legal Location UTM Coordin...,-1
...,...,...,...
28886,TABLE 1 GENERAL LAND USE AND ENVIRONMENTAL SET...,Setting Information Project Details Total Len...,-1
28887,TABLE 2 OCCURRENCES OF SPECIES WITH SPECIAL CO...,Species Common Name Scientific Name Federal D...,-1
28888,TABLE 2 OCCURRENCES OF SPECIES WITH SPECIAL CO...,Species Common Name Scientific Name Federal D...,-1
28889,TABLE 2 OCCURRENCES OF SPECIES WITH SPECIAL CO...,Species Common Name Scientific Name Federal D...,-1
